In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.constraints import max_norm


In [2]:
my_gpu = tf.config.experimental.list_physical_devices('GPU')
print(my_gpu)
tf.config.experimental.set_memory_growth(my_gpu[0],True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
df = pd.read_csv('final_data.csv')

df.drop(['href', 'href','stress_address','city','state','county','img_href',"zip"], axis=1, inplace=True)
dfX = df.drop("price",axis=1)
dfX

,beds,bath,area,year_built,lot_size,walk_score,transit_score,bike_score,cooling,heating,has_pool,crime_rate_per_100000,minimum_wage,college_completion,unemployment_rate,median_household_income
0,3,2.0,1229,1983,0,68,39,63,0,0,0,442.965072,7.25,0.315,7.7,61807.0
1,3,1.0,1348,1913,0,70,0,46,0,0,0,102.396264,7.25,0.413,8.3,93587.0
2,3,1.0,1434,1941,0,45,19,39,1,2,0,177.018370,7.25,0.437,6.5,68975.0
3,3,2.0,1356,1961,0,11,15,33,2,2,0,721.521934,7.25,0.290,9.3,44773.0
4,5,1.5,2266,1940,0,5,0,42,0,3,0,18.351570,8.70,0.097,3.9,62596.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36330,2,1.0,928,1962,0,47,0,35,0,3,0,89.855758,11.25,0.194,8.2,52543.0
36331,3,1.5,1454,1956,0,42,0,44,0,0,0,290.721821,9.65,0.249,12.0,65051.0
36332,4,2.5,1772,2006,0,1,24,35,1,2,0,738.481892,9.00,0.344,8.1,56148.0
36333,3,1.5,1706,1930,0,55,0,46,0,3,0,521.234940,8.70,0.325,10.4,52503.0


In [4]:
X,y = np.array(dfX),np.array(df["price"])
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [5]:
model = Sequential([
    Dense(units=128,input_shape=(len(X[0]),),kernel_initializer=initializers.RandomNormal(stddev=0.01),activation = LeakyReLU(alpha=0.01)),
    Dense(units=256,kernel_initializer=initializers.RandomNormal(stddev=0.01),activation = LeakyReLU(alpha=0.01)),
    Dense(units=256,kernel_initializer=initializers.RandomNormal(stddev=0.01),activation = LeakyReLU(alpha=0.01)),
    Dense(units=256,kernel_initializer=initializers.RandomNormal(stddev=0.01),activation = LeakyReLU(alpha=0.01)),
    Dense(units=1,kernel_initializer=initializers.RandomNormal(stddev=0.01),activation =  LeakyReLU(alpha=0.01))
])

In [6]:
def coeff_determination(y_true, y_pred):
    from tensorflow.keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [7]:
model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_absolute_error', metrics = [coeff_determination])
convergeAlarm = EarlyStopping(monitor='loss', min_delta=0, patience=50)
def lr_scheduler(epoch, lr):
    if epoch % 15 == 0 and epoch:
        return lr /1.15
    return lr

In [8]:

history = model.fit(x=X_train, y=y_train, batch_size=128, epochs=40000, verbose=2,validation_split=0.2,shuffle=True, callbacks=[convergeAlarm])

Epoch 1/40000
182/182 - 1s - loss: 354999.9688 - coeff_determination: 0.1247 - val_loss: 320548.6875 - val_coeff_determination: -5.7310e-01
Epoch 2/40000
182/182 - 0s - loss: 297292.4688 - coeff_determination: 0.2780 - val_loss: 327262.2812 - val_coeff_determination: -3.4284e+00
Epoch 3/40000
182/182 - 0s - loss: 289663.9062 - coeff_determination: 0.0101 - val_loss: 352371.6875 - val_coeff_determination: -1.5027e+01
Epoch 4/40000
182/182 - 0s - loss: 284410.0312 - coeff_determination: 0.2382 - val_loss: 361576.0312 - val_coeff_determination: -2.6324e+01
Epoch 5/40000
182/182 - 0s - loss: 282130.0625 - coeff_determination: 0.1661 - val_loss: 423909.2188 - val_coeff_determination: -5.9208e+01
Epoch 6/40000
182/182 - 0s - loss: 281334.7188 - coeff_determination: -2.0967e-01 - val_loss: 384998.4688 - val_coeff_determination: -5.3647e+01
Epoch 7/40000
182/182 - 0s - loss: 277293.5625 - coeff_determination: -8.4054e-01 - val_loss: 387325.5000 - val_coeff_determination: -6.0070e+01
Epoch 8/40

Epoch 61/40000
182/182 - 1s - loss: 208399.9688 - coeff_determination: 0.6391 - val_loss: 256101.2031 - val_coeff_determination: 0.3761
Epoch 62/40000
182/182 - 1s - loss: 207684.4688 - coeff_determination: 0.6527 - val_loss: 260264.2031 - val_coeff_determination: 0.3433
Epoch 63/40000
182/182 - 1s - loss: 206508.8125 - coeff_determination: 0.6465 - val_loss: 262805.4688 - val_coeff_determination: 0.2512
Epoch 64/40000
182/182 - 1s - loss: 204364.4062 - coeff_determination: 0.6467 - val_loss: 260551.8594 - val_coeff_determination: 0.3435
Epoch 65/40000
182/182 - 1s - loss: 207205.0781 - coeff_determination: 0.6388 - val_loss: 259527.6406 - val_coeff_determination: 0.3887
Epoch 66/40000
182/182 - 1s - loss: 201885.2969 - coeff_determination: 0.6690 - val_loss: 262127.1875 - val_coeff_determination: 0.2962
Epoch 67/40000
182/182 - 1s - loss: 201598.4062 - coeff_determination: 0.6646 - val_loss: 254560.0938 - val_coeff_determination: 0.4101
Epoch 68/40000
182/182 - 1s - loss: 201701.4062 

Epoch 121/40000
182/182 - 0s - loss: 162800.5469 - coeff_determination: 0.7543 - val_loss: 253733.3906 - val_coeff_determination: 0.4954
Epoch 122/40000
182/182 - 0s - loss: 164487.4062 - coeff_determination: 0.7671 - val_loss: 263174.6562 - val_coeff_determination: 0.3143
Epoch 123/40000
182/182 - 0s - loss: 158818.3438 - coeff_determination: 0.7842 - val_loss: 257482.7812 - val_coeff_determination: 0.4130
Epoch 124/40000
182/182 - 0s - loss: 160272.9375 - coeff_determination: 0.7585 - val_loss: 260322.6562 - val_coeff_determination: 0.2773
Epoch 125/40000
182/182 - 0s - loss: 158309.0469 - coeff_determination: 0.7821 - val_loss: 287000.6250 - val_coeff_determination: -1.0177e+00
Epoch 126/40000
182/182 - 0s - loss: 159556.7656 - coeff_determination: 0.7634 - val_loss: 256578.2656 - val_coeff_determination: 0.3696
Epoch 127/40000
182/182 - 0s - loss: 159965.2969 - coeff_determination: 0.7686 - val_loss: 262407.6875 - val_coeff_determination: -1.5528e-02
Epoch 128/40000
182/182 - 0s - 

Epoch 181/40000
182/182 - 0s - loss: 130378.6953 - coeff_determination: 0.8503 - val_loss: 265706.2188 - val_coeff_determination: 0.3219
Epoch 182/40000
182/182 - 0s - loss: 124456.4609 - coeff_determination: 0.8551 - val_loss: 275244.9375 - val_coeff_determination: 0.1607
Epoch 183/40000
182/182 - 0s - loss: 128004.5781 - coeff_determination: 0.8534 - val_loss: 266344.0625 - val_coeff_determination: 0.1895
Epoch 184/40000
182/182 - 0s - loss: 126031.0625 - coeff_determination: 0.8554 - val_loss: 269962.6250 - val_coeff_determination: -6.2892e-02
Epoch 185/40000
182/182 - 0s - loss: 126486.4062 - coeff_determination: 0.8596 - val_loss: 259278.3125 - val_coeff_determination: 0.4817
Epoch 186/40000
182/182 - 0s - loss: 127019.6250 - coeff_determination: 0.8436 - val_loss: 270223.6875 - val_coeff_determination: 0.1801
Epoch 187/40000
182/182 - 0s - loss: 122191.8672 - coeff_determination: 0.8667 - val_loss: 276132.8125 - val_coeff_determination: -1.7168e-02
Epoch 188/40000
182/182 - 0s - 

Epoch 241/40000
182/182 - 1s - loss: 101628.4531 - coeff_determination: 0.8974 - val_loss: 279281.5938 - val_coeff_determination: -4.3891e-02
Epoch 242/40000
182/182 - 1s - loss: 103763.7578 - coeff_determination: 0.8970 - val_loss: 275646.3125 - val_coeff_determination: 0.2416
Epoch 243/40000
182/182 - 1s - loss: 104342.1953 - coeff_determination: 0.8879 - val_loss: 278745.0625 - val_coeff_determination: 0.2268
Epoch 244/40000
182/182 - 1s - loss: 103107.8047 - coeff_determination: 0.8950 - val_loss: 280269.3125 - val_coeff_determination: 0.0289
Epoch 245/40000
182/182 - 1s - loss: 101865.3203 - coeff_determination: 0.8930 - val_loss: 274852.5312 - val_coeff_determination: 0.3076
Epoch 246/40000
182/182 - 0s - loss: 103045.6719 - coeff_determination: 0.8867 - val_loss: 278181.0938 - val_coeff_determination: 0.0724
Epoch 247/40000
182/182 - 1s - loss: 101284.6953 - coeff_determination: 0.8995 - val_loss: 272150.5312 - val_coeff_determination: 0.3026
Epoch 248/40000
182/182 - 0s - loss:

...

182/182 - 2s - loss: 38531.1523 - coeff_determination: 0.9833 - val_loss: 291088.3750 - val_coeff_determination: 0.2120
Epoch 787/40000
182/182 - 2s - loss: 38254.3398 - coeff_determination: 0.9831 - val_loss: 293881.0000 - val_coeff_determination: 0.2229
Epoch 788/40000
182/182 - 2s - loss: 39689.1367 - coeff_determination: 0.9858 - val_loss: 290776.6875 - val_coeff_determination: 0.1721
Epoch 789/40000
182/182 - 2s - loss: 38882.9297 - coeff_determination: 0.9852 - val_loss: 291021.3750 - val_coeff_determination: 0.2018
Epoch 790/40000
182/182 - 2s - loss: 38646.0664 - coeff_determination: 0.9837 - val_loss: 289919.1562 - val_coeff_determination: 0.2484
Epoch 791/40000
182/182 - 2s - loss: 38576.2500 - coeff_determination: 0.9834 - val_loss: 292528.2500 - val_coeff_determination: 0.1046
Epoch 792/40000
182/182 - 2s - loss: 38528.6875 - coeff_determination: 0.9855 - val_loss: 290980.1250 - val_coeff_determination: 0.1807
Epoch 793/40000
182/182 - 2s - loss: 38796.5508 - coeff_determin

Epoch 847/40000
182/182 - 3s - loss: 37589.3750 - coeff_determination: 0.9865 - val_loss: 293292.8438 - val_coeff_determination: 0.1932
Epoch 848/40000
182/182 - 3s - loss: 38309.6602 - coeff_determination: 0.9843 - val_loss: 285861.3438 - val_coeff_determination: 0.2082
Epoch 849/40000
182/182 - 2s - loss: 36790.0664 - coeff_determination: 0.9844 - val_loss: 296085.6250 - val_coeff_determination: 0.0790
Epoch 850/40000
182/182 - 4s - loss: 36778.3438 - coeff_determination: 0.9858 - val_loss: 292800.2812 - val_coeff_determination: 0.1088
Epoch 851/40000
182/182 - 3s - loss: 39840.6016 - coeff_determination: 0.9806 - val_loss: 288370.8750 - val_coeff_determination: 0.2930
Epoch 852/40000
182/182 - 3s - loss: 37552.8750 - coeff_determination: 0.9835 - val_loss: 289519.7188 - val_coeff_determination: 0.1220
Epoch 853/40000
182/182 - 2s - loss: 36182.9180 - coeff_determination: 0.9848 - val_loss: 291249.6562 - val_coeff_determination: 0.2150
Epoch 854/40000
182/182 - 2s - loss: 36864.4609 

Epoch 908/40000
182/182 - 2s - loss: 36565.5430 - coeff_determination: 0.9846 - val_loss: 292978.8750 - val_coeff_determination: 0.1623
Epoch 909/40000
182/182 - 2s - loss: 35396.3828 - coeff_determination: 0.9861 - val_loss: 291442.1562 - val_coeff_determination: 0.0928
Epoch 910/40000
182/182 - 2s - loss: 33488.8047 - coeff_determination: 0.9888 - val_loss: 286926.5625 - val_coeff_determination: 0.2207
Epoch 911/40000
182/182 - 2s - loss: 34959.4141 - coeff_determination: 0.9875 - val_loss: 288902.2500 - val_coeff_determination: 0.1895
Epoch 912/40000
182/182 - 2s - loss: 34692.4180 - coeff_determination: 0.9851 - val_loss: 291341.8750 - val_coeff_determination: 0.1575
Epoch 913/40000
182/182 - 2s - loss: 34399.0352 - coeff_determination: 0.9887 - val_loss: 288596.7500 - val_coeff_determination: 0.2257
Epoch 914/40000
182/182 - 2s - loss: 34897.2188 - coeff_determination: 0.9858 - val_loss: 285403.7188 - val_coeff_determination: 0.2609
Epoch 915/40000
182/182 - 2s - loss: 36678.7578 

Epoch 969/40000
182/182 - 2s - loss: 33682.8594 - coeff_determination: 0.9866 - val_loss: 288780.4688 - val_coeff_determination: 0.2424
Epoch 970/40000
182/182 - 2s - loss: 34881.7148 - coeff_determination: 0.9867 - val_loss: 290384.0625 - val_coeff_determination: 0.0642
Epoch 971/40000
182/182 - 2s - loss: 34047.2422 - coeff_determination: 0.9860 - val_loss: 282550.5000 - val_coeff_determination: 0.2896
Epoch 972/40000
182/182 - 2s - loss: 35095.4297 - coeff_determination: 0.9888 - val_loss: 285001.5625 - val_coeff_determination: 0.2352
Epoch 973/40000
182/182 - 2s - loss: 33143.5469 - coeff_determination: 0.9841 - val_loss: 289895.1250 - val_coeff_determination: 0.1292
Epoch 974/40000
182/182 - 2s - loss: 34556.4336 - coeff_determination: 0.9876 - val_loss: 290169.7812 - val_coeff_determination: 0.1585
Epoch 975/40000
182/182 - 3s - loss: 32795.2461 - coeff_determination: 0.9873 - val_loss: 293042.4375 - val_coeff_determination: 0.0481
Epoch 976/40000
182/182 - 3s - loss: 34287.2539 

Epoch 1029/40000
182/182 - 2s - loss: 31356.0391 - coeff_determination: 0.9887 - val_loss: 286289.8125 - val_coeff_determination: 0.2091
Epoch 1030/40000
182/182 - 2s - loss: 31265.1914 - coeff_determination: 0.9905 - val_loss: 285868.7812 - val_coeff_determination: 0.2719
Epoch 1031/40000
182/182 - 2s - loss: 32967.2578 - coeff_determination: 0.9889 - val_loss: 292205.1875 - val_coeff_determination: 0.1402
Epoch 1032/40000
182/182 - 2s - loss: 32748.1289 - coeff_determination: 0.9894 - val_loss: 290671.8125 - val_coeff_determination: 0.0638
Epoch 1033/40000
182/182 - 1s - loss: 32358.7949 - coeff_determination: 0.9906 - val_loss: 283646.1875 - val_coeff_determination: 0.1746
Epoch 1034/40000
182/182 - 1s - loss: 32133.0879 - coeff_determination: 0.9884 - val_loss: 284903.2812 - val_coeff_determination: 0.1858
Epoch 1035/40000
182/182 - 2s - loss: 32118.7773 - coeff_determination: 0.9900 - val_loss: 290481.1250 - val_coeff_determination: 0.1965
Epoch 1036/40000
182/182 - 2s - loss: 311